In [1]:
from __future__ import division

from pyqtgraph.Qt import QtGui, QtCore
import numpy as np
from scipy.signal import filtfilt
from numpy import nonzero, diff

import pyqtgraph as pg

#!/usr/bin/env python

from recorder import SoundCardDataSource
w = np.load('pretrained.npz')['w']
b = np.load('pretrained.npz')['b']

act_limit=3
pg.setConfigOption('background', 'w')
pg.setConfigOption('foreground', 'k')
# Based on function from numpy 1.8
def rfftfreq(n, d=1.0):
    """
    Return the Discrete Fourier Transform sample frequencies
    (for usage with rfft, irfft).

    The returned float array `f` contains the frequency bin centers in cycles
    per unit of the sample spacing (with zero at the start). For instance, if
    the sample spacing is in seconds, then the frequency unit is cycles/second.

    Given a window length `n` and a sample spacing `d`::

    f = [0, 1, ..., n/2-1, n/2] / (d*n) if n is even
    f = [0, 1, ..., (n-1)/2-1, (n-1)/2] / (d*n) if n is odd

    Unlike `fftfreq` (but like `scipy.fftpack.rfftfreq`)
    the Nyquist frequency component is considered to be positive.

    Parameters
    ----------
    n : int
    Window length.
    d : scalar, optional
    Sample spacing (inverse of the sampling rate). Defaults to 1.

    Returns
    -------
    f : ndarray
    Array of length ``n//2 + 1`` containing the sample frequencies.
    """
    if not isinstance(n, int):
        raise ValueError("n should be an integer")
    val = 1.0/(n*d)
    N = n//2 + 1
    results = np.arange(0, N, dtype=int)
    return results * val


def fft_slices(x):
    Nslices, Npts = x.shape
    window = np.hanning(Npts)

    # Calculate FFT
    fx = np.fft.rfft(window[np.newaxis, :] * x, axis=1)

    # Convert to normalised PSD
    Pxx = abs(fx)**2 / (np.abs(window)**2).sum()

    # Scale for one-sided (excluding DC and Nyquist frequencies)
    Pxx[:, 1:-1] *= 2

    # And scale by frequency to get a result in (dB/Hz)
    # Pxx /= Fs
    return Pxx ** 0.5


def find_peaks(Pxx):
    # filter parameters
    b, a = [0.01], [1, -0.99]
    Pxx_smooth = filtfilt(b, a, abs(Pxx))
    peakedness = abs(Pxx) / Pxx_smooth

    # find peaky regions which are separated by more than 10 samples
    peaky_regions = nonzero(peakedness > 1)[0]
    edge_indices = nonzero(diff(peaky_regions) > 10)[0]  # RH edges of peaks
    edges = [0] + [(peaky_regions[i] + 5) for i in edge_indices]
    if len(edges) < 2:
        edges += [len(Pxx) - 1]

    peaks = []
    for i in range(len(edges) - 1):
        j, k = edges[i], edges[i+1]
        peaks.append(j + np.argmax(peakedness[j:k]))
    return peaks


def fft_buffer(x):
    window = np.hanning(x.shape[0])

    # Calculate FFT
    fx = np.fft.rfft(window * x)

    # Convert to normalised PSD
    Pxx = abs(fx)**2 / (np.abs(window)**2).sum()

    # Scale for one-sided (excluding DC and Nyquist frequencies)
    Pxx[1:-1] *= 2

    # And scale by frequency to get a result in (dB/Hz)
    # Pxx /= Fs
    return Pxx ** 0.5


class LiveFFTWindow(pg.GraphicsWindow):
    def __init__(self, recorder):
        super(LiveFFTWindow, self).__init__(title="Live FFT")
        self.recorder = recorder
        self.paused = False
        self.logScale = False
        self.showPeaks = False
        self.downsample = True

        # Setup plots
        self.p1 = self.addPlot()
        self.p1.setLabel('bottom', 'Time', 's')
        self.p1.setLabel('left', 'Amplitude')
        self.p1.setTitle("")
        self.ts = self.p1.plot(pen='b')
        self.nextRow()
        self.p2 = self.addPlot()
        self.p2.setLabel('bottom', 'Frequency', 'Hz')
        self.spec = self.p2.plot(pen=(0, 0, 200),
                                 brush=(0, 0, 200),
                                 fillLevel=-100)
        self.nextRow()
        self.p3 = self.addPlot()
        self.p3.setLabel('bottom', 'Frequency', 'Hz')
        self.p3.setLabel('left', 'Weight', '')
        self.p3.setTitle('Perceptron weights')
        self.weight = self.p3.plot(pen='b', brush='b')

        self.nextRow()
        self.p4 = self.addPlot()
        self.p4.setLabel('bottom', 'INPUT', '')
        self.p4.setLabel('left', 'OUTPUT', '')
        self.p4.setTitle('Activation')
        self.act = self.p4.plot(pen='b', brush='b')

        self.nextRow()
        self.p5 = self.addPlot()
        self.p5.setTitle('Non-Linearity')

        x = np.linspace(-act_limit, act_limit, 1000)
        y = np.tanh(x)
        self.p5.plot(pen='r', brush='r').setData(x=x, y=y)

        #notePen = pg.mkPen((220, 0, 50, 100))
        #self.act_line = self.p4.addLine(x=0, pen=notePen)

        '''
        # Show note lines
            A = 440.0
            notePen = pg.mkPen((0, 200, 50, 100))
            while A < (self.recorder.fs / 2):
                self.p2.addLine(x=A, pen=notePen)
                A *= 2
        '''

        # Lines for marking peaks
        self.peakMarkers = []

        # Data ranges
        self.resetRanges()

        # Timer to update plots
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update)
        interval_ms = 1000 * (self.recorder.chunk_size / self.recorder.fs)
        #print ("Updating graphs every %.1f ms" % interval_ms)
        self.timer.start(interval_ms)

    def resetRanges(self):
        self.timeValues = self.recorder.timeValues
        self.freqValues = rfftfreq(len(self.timeValues),
                                   1./self.recorder.fs)

        self.p1.setRange(xRange=(0, self.timeValues[-1]), yRange=(-1, 1))
        if self.logScale:
            self.p2.setRange(xRange=(0, self.freqValues[-1]),
                             yRange=(-60, 20))
            self.spec.setData(fillLevel=-100)
            self.p2.setLabel('left', 'PSD', 'dB / Hz')
        else:
            self.p2.setRange(xRange=(0, self.freqValues[-1]),
                             yRange=(0, 50))
            self.spec.setData(fillLevel=0)
            self.p2.setLabel('left', 'PSD', '1 / Hz')

    def plotPeaks(self, Pxx):
        # find peaks bigger than a certain threshold
        peaks = [p for p in find_peaks(Pxx) if Pxx[p] > 0.3]

        if self.logScale:
            Pxx = 20*np.log10(Pxx)

        # Label peaks
        old = self.peakMarkers
        self.peakMarkers = []
        for p in peaks:
            if old:
                t = old.pop()
            else:
                t = pg.TextItem(color=(150, 150, 150, 150))
                self.p2.addItem(t)
            self.peakMarkers.append(t)
            t.setText("%.1f Hz" % self.freqValues[p])
            t.setPos(self.freqValues[p], Pxx[p])
        for t in old:
            self.p2.removeItem(t)
            del t

    def update(self):
        if self.paused:
            return
        data = self.recorder.get_buffer()
        weighting = np.exp(self.timeValues / self.timeValues[-1])
        Pxx = fft_buffer(weighting * data[:, 0])

        if self.downsample:
            downsample_args = dict(autoDownsample=False,
                                   downsampleMethod='subsample',
                                   downsample=10)
        else:
            downsample_args = dict(autoDownsample=True)

        self.ts.setData(x=self.timeValues, y=data[:, 0], **downsample_args)
        self.spec.setData(x=self.freqValues,
                          y=(20*np.log10(Pxx) if self.logScale else Pxx))

        pxx = (Pxx - Pxx.min()) 
        pxx /= pxx.max()
        # Show note lines


        notePen = pg.mkPen((0, 200, 50, 100))
        a = np.dot(w, Pxx) + b
        self.weight.setData(x=self.freqValues, y=w)
        print(a)
        a = max(-act_limit, min(a, act_limit))
        x = np.linspace(-act_limit, act_limit, len(w))
        y = np.tanh(x)
        A = np.ones(len(x))*0

        idx = (np.abs(x-a)).argmin()
        #print(idx)
        if idx == 0:
            A[5:15] = 1
        elif idx == len(A) - 1:
            A[-30:-10] = 1
        else:
            A[idx-10: idx+10] = 1
        self.act.setData(x=x, y=A)
        if self.showPeaks:
            self.plotPeaks(Pxx)

    def keyPressEvent(self, event):
        text = event.text()
        if text == " ":
            self.paused = not self.paused
            self.p1.setTitle("PAUSED" if self.paused else "")
        elif text == "l":
            self.logScale = not self.logScale
            self.resetRanges()
        elif text == "d":
            self.downsample = not self.downsample
        elif text == "+":
            self.recorder.num_chunks *= 2
            self.resetRanges()
        elif text == "-":
            self.recorder.num_chunks /= 2
            self.resetRanges()
        elif text == "p":
            self.showPeaks = not self.showPeaks
        else:
            super(LiveFFTWindow, self).keyPressEvent(event)

        # Setup plots
#QtGui.QApplication.setGraphicsSystem('opengl')
app = QtGui.QApplication([])
#pg.setConfigOptions(antialias=True)

# Setup recorder
#FS = 12000
#FS = 22000
FS = 44000
recorder = SoundCardDataSource(num_chunks=3,
                               sampling_rate=FS,
                               chunk_size=4*1024)
win = LiveFFTWindow(recorder)
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

[-1.89198112]
[-1.89198112]
[-2.60355282]
[-2.3973856]
[-2.3973856]
[-2.59328604]
[-2.75625515]
[-2.75625515]
[-2.42822838]
[-2.42856526]
[-2.42856526]
[-2.37307596]
[-2.65693927]
[-2.65693927]
[-2.43428564]
[-2.64271307]
[-2.64271307]
[-2.4861145]
[-2.61197424]
[-2.61197424]
[-2.65336561]
[-2.65336561]
[-2.7283051]
[-2.65325356]
[-2.38701177]
[-2.51409793]
[-2.51409793]
[-2.50918913]
[-2.50918913]
[-2.52202606]
[-2.61970973]
[-3.18079257]
[-3.18079257]
[-3.15245867]
[-2.77063847]
[-2.77063847]
[-2.76560569]
[-2.73143387]
[-2.63138437]
[-2.63138437]
[-2.64618635]
[-2.40088892]
[-2.4803195]
[-2.4803195]
[-2.65688229]
[-6.41883421]
[-6.41883421]
[-9.97015762]
[-6.3911891]
[-4.54494858]
[-4.54494858]
[-4.16361618]
[-4.16361618]
[-2.88082123]
[-2.78369784]
[-2.78369784]
[-2.71888638]
[-2.71888638]
[-2.96080542]
[-2.96080542]
[-2.73541951]
[-3.01062536]
[-3.01062536]
[-2.95995283]
[-2.63104439]
[-2.79557776]
[-2.79557776]
[-7.53087044]
[-2.73458338]
[-2.73458338]
[-2.65111589]
[-2.65111589]

[-2.15720081]
[-2.15720081]
[-2.30245447]
[-2.30245447]
[-2.55334163]
[-2.26841736]
[-2.59388638]
[-2.59388638]
[-2.40998769]
[-2.23352122]
[-2.23352122]
[-2.2297802]
[-2.21686482]
[-2.37387085]
[-2.37387085]
[-2.39341688]
[-2.35513926]
[-2.35513926]
[-2.17718124]
[-2.11959243]
[-2.21345782]
[-2.21345782]
[-2.31455827]
[-2.37096477]
[-2.37096477]
[-2.34308577]
[-2.3823092]
[-2.3823092]
[-2.32452035]
[-2.27087116]
[-2.27087116]
[-2.34963536]
[-2.28280425]
[-2.28280425]
[-2.4267087]
[-2.36295414]
[-2.67488861]
[-2.67488861]
[-2.74456716]
[-2.74456716]
[-2.42560077]
[-2.48906326]
[-2.61970687]
[-2.61970687]
[-2.47580147]
[-2.51763821]
[-2.30835056]
[-2.37272096]
[-2.37272096]
[-2.40144014]
[-2.40144014]
[-2.36724305]
[-2.54863238]
[-2.29846597]
[-2.29846597]
[-2.30887103]
[-2.38083649]
[-2.38083649]
[-2.51758552]
[-2.84058237]
[-2.85159922]
[-2.85159922]
[-2.44596171]
[-2.47160673]
[-2.47160673]
[-2.31329155]
[-2.27577782]
[-2.38516331]
[-2.38516331]
[-2.41621661]
[-2.41621661]
[-2.327889

[-2.14452314]
[-2.14452314]
[-2.14582753]
[-2.24774456]
[-2.24774456]
[-2.39688873]
[-2.39688873]
[-2.22209024]
[-2.36372256]
[-2.36372256]
[-2.38682747]
[-2.29730153]
[-2.29730153]
[-2.33811021]
[-2.16513658]
[-2.16513658]
[-2.20348668]
[-2.39278579]
[-2.20928431]
[-2.20928431]
[-2.23227453]
[-2.23227453]
[-2.26954603]
[-2.26149774]
[-2.27916956]
[-2.27916956]
[-2.3254025]
[-2.3999896]
[-2.3999896]
[-2.49598479]
[-2.40778184]
[-2.40778184]
[-2.30933952]
[-2.38399029]
[-2.38399029]
[-2.58846426]
[-2.58846426]
[-2.33564234]
[-2.29489946]
[-2.29869986]
[-2.29869986]
[-2.33868074]
[-2.33868074]
[-2.28296781]
[-2.35206556]
[-2.35206556]
[-2.35483003]
[-2.31978559]
[-2.31978559]
[-2.36126971]
[-2.2432456]
[-2.31800747]
[-2.31800747]
[-2.19893837]
[-2.46028662]
[-2.46028662]
[-2.41425085]
[-2.32425451]
[-2.28902411]
[-2.28902411]
[-2.33731103]
[-2.33731103]
[-2.45203519]
[-2.18696904]
[-2.29593682]
[-2.29593682]
[-2.39789176]
[-2.41619492]
[-2.41619492]
[-2.28580165]
[-2.38646173]
[-2.341057

[-3.27804255]
[-2.94814396]
[-2.75788021]
[-2.75788021]
[-3.01933479]
[-3.2741785]
[-2.80119467]
[-2.80119467]
[-2.66285348]
[-3.15454745]
[-3.15454745]
[-2.79387617]
[-2.79387617]
[-2.85062814]
[-2.85062814]
[-2.63434649]
[-2.63434649]
[-2.73122454]
[-2.86643362]
[-2.86643362]
[-2.78500652]
[-2.78500652]
[-2.40581846]
[-2.44258857]
[-2.36444473]
[-2.36444473]
[-2.48538256]
[-2.49205089]
[-2.32058835]
[-2.32058835]
[-2.46524692]
[-2.53120947]
[-2.53120947]
[-2.34862852]
[-2.50302553]
[-2.69599652]
[-2.69599652]
[-2.45215797]
[-2.43394661]
[-2.43394661]
[-2.47004032]
[-2.38477278]
[-2.30649281]
[-2.30649281]
[-2.3343327]
[-2.52582407]
[-2.6135149]
[-2.6135149]
[-2.39690351]
[-2.42091131]
[-2.42091131]
[-2.5393846]
[-2.5393846]
[-2.63999438]
[-2.63999438]
[-3.10581613]
[-2.70883203]
[-2.70883203]
[-2.6943388]
[-2.6943388]
[-2.65371799]
[-2.65371799]
[-2.66870785]
[-2.66870785]
[-2.54012394]
[-2.39110351]
[-2.39110351]
[-2.30246735]
[-2.75433064]
[-2.80861712]
[-2.80861712]
[-2.43609715]


[-2.29660535]
[-2.29660535]
[-2.33577013]
[-2.37743235]
[-2.29361844]
[-2.29361844]
[-2.16594148]
[-2.38509536]
[-2.38509536]
[-2.2878623]
[-2.31573606]
[-2.15402937]
[-2.15402937]
[-2.37677002]
[-2.37677002]
[-2.32700968]
[-2.43283463]
[-2.39331985]
[-2.39331985]
[-2.31149483]
[-2.28783965]
[-2.24404502]
[-2.24404502]
[-2.32480907]
[-2.32480907]
[-2.441571]
[-2.21653771]
[-2.21653771]
[-2.42013788]
[-2.20712519]
[-2.20712519]
[-2.39065075]
[-2.54955959]
[-2.19611716]
[-2.19611716]
[-2.19010496]
[-2.49010658]
[-2.49010658]
[-2.52359724]
[-2.37530231]
[-2.42018747]
[-2.42018747]
[-2.50768042]
[-2.25120926]
[-2.25120926]
[-2.45366716]
[-2.45366716]
[-2.31762767]
[-2.89391685]
[-2.89391685]
[-2.9500761]
[-2.9500761]
[-2.80437374]
[-2.46499991]
[-2.46499991]
[-2.6275425]
[-2.74994516]
[-2.59156466]
[-2.59156466]
[-2.57173777]
[-2.56454563]
[-2.56454563]
[-2.61968851]
[-2.74364781]
[-2.72710037]
[-2.72710037]
[-2.45557308]
[-2.60617304]
[-2.60617304]
[-2.97358418]
[-2.97358418]
[-3.05091286